# Завдання 1

Для кожної із адміністративних одиниць України завантажити тестові структуровані файли, що містять значення VHI-індексу.
Ця процедура має бути автоматизована, параметром процедури має бути індекс (номер) області.
При зберіганні файлу до його імені потрібно додати дату та час завантаження.

Передбачити повторні запуски скрипту, довантаження нових даних та колізію
даних;

In [8]:
import urllib.request
import hashlib
import os
from datetime import datetime

# Папка для збереження CSV-файлів
DATA_DIR = "CSV_Files"
if not os.path.exists(DATA_DIR):
    os.makedirs(DATA_DIR)

# Функція для обчислення хешу файлу (SHA-256)
def compute_sha256(file_path):
    """Обчислює SHA-256 хеш файлу для перевірки дублікатів."""
    sha256 = hashlib.sha256()
    with open(file_path, "rb") as f:
        for chunk in iter(lambda: f.read(4096), b""):
            sha256.update(chunk)
    return sha256.hexdigest()

# Функція для перевірки коректності відповіді сервера
def is_valid_response(response):
    """Перевіряє, чи отримані коректні дані (не HTML-сторінка з помилкою)."""
    content_type = response.getheader('Content-Type', '')
    return 'text/plain' in content_type or 'csv' in content_type

# Функція для завантаження VHI-даних для конкретної області
def download_vhi_data(region_ID):
    """Завантажує VHI-дані для області та зберігає у файлі з унікальним ім'ям."""

    # Формування URL для отримання даних
    url = f"https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={region_ID}&year1=1981&year2=2024&type=Mean"

    # Генерація унікальної назви файлу із зазначенням часу
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_file_path = os.path.join(DATA_DIR, f"VHI_{region_ID}_{timestamp}.csv")

    try:
        # Відправка запиту та отримання даних
        response = urllib.request.urlopen(url)

        content = response.read()

        # Запис отриманих даних у CSV-файл
        with open(output_file_path, 'wb') as file:
            file.write(content)

        # Перевірка наявності дубліката
        for stored_file in os.listdir(DATA_DIR):
            if stored_file.endswith(".csv") and f"Region{region_ID}" in stored_file:
                stored_path = os.path.join(DATA_DIR, stored_file)

                # Якщо файл ідентичний, видаляємо новий дублікат
                if stored_path != output_file_path and os.path.exists(stored_path):
                    if compute_sha256(stored_path) == compute_sha256(output_file_path):
                        print(f"Файл для області {region_ID} вже є. Видаляю дубль.")
                        os.remove(output_file_path)
                        return

        print(f"Файл для області {region_ID} успішно завантажено: {output_file_path}")

    except Exception as err:
        print(f"Помилка при отриманні даних для області {region_ID}: {err}")

# Завантаження даних для всіх 27 адміністративних одиниць України (25 областей + АР Крим + Київ)
for region_ID in range(1, 28):
    download_vhi_data(region_ID)

Файл для області 1 успішно завантажено: CSV_Files\VHI_1_20250328_160402.csv
Файл для області 2 успішно завантажено: CSV_Files\VHI_2_20250328_160405.csv
Файл для області 3 успішно завантажено: CSV_Files\VHI_3_20250328_160407.csv
Файл для області 4 успішно завантажено: CSV_Files\VHI_4_20250328_160408.csv
Файл для області 5 успішно завантажено: CSV_Files\VHI_5_20250328_160411.csv
Файл для області 6 успішно завантажено: CSV_Files\VHI_6_20250328_160414.csv
Файл для області 7 успішно завантажено: CSV_Files\VHI_7_20250328_160416.csv
Файл для області 8 успішно завантажено: CSV_Files\VHI_8_20250328_160418.csv
Файл для області 9 успішно завантажено: CSV_Files\VHI_9_20250328_160419.csv
Файл для області 10 успішно завантажено: CSV_Files\VHI_10_20250328_160422.csv
Файл для області 11 успішно завантажено: CSV_Files\VHI_11_20250328_160423.csv
Файл для області 12 успішно завантажено: CSV_Files\VHI_12_20250328_160426.csv
Файл для області 13 успішно завантажено: CSV_Files\VHI_13_20250328_160427.csv
Файл

# Завдання 2

Зчитати завантажені текстові файли у фрейм
(https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html) (детальніше
про роботу із фреймами буде розказано у подальших лабораторних роботах).
Імена стовбців фрейму мають бути змістовними та легкими для сприйняття (не
повинно бути спеціалізованих символів, пробілів тощо). Ця задача має бути
реалізована у вигляді окремої процедури, яка на вхід приймає шлях до
директорії, в якій зберігаються файли;

In [9]:
import pandas as pd
import os
import re

# Папка з CSV-файлами та вихідний файл
DATA_DIR = "CSV_Files"  # Твоя директорія
output_path = "full.csv" # Файл для збереження

# Імена колонок для зчитування
COLUMN_NAMES = ["Year", "Week", "SMN", "SMT", "VCI", "TCI", "VHI", "PROVINCE_ID"]

# Порожній DataFrame для об'єднання даних
combined_data = pd.DataFrame(columns=COLUMN_NAMES)

# Отримуємо список CSV-файлів, відсортованих за номером області
files = sorted([f for f in os.listdir(DATA_DIR) if f.endswith('.csv')],
               key=lambda x: int(re.findall(r'\d+', x)[0]))

# Обробка кожного файлу
for file_name in files:
    file_path = os.path.join(DATA_DIR, file_name)

    try:
        # Отримуємо ID області з назви файлу
        province_id = int(re.findall(r'\d+', file_name)[0])

        # Зчитуємо файл, пропускаючи перші 2 рядки
        df = pd.read_csv(file_path, skiprows=2, names=COLUMN_NAMES)

        # Видаляємо HTML-теги з Year
        df["Year"] = df["Year"].astype(str).str.replace(r'<tt><pre>|</pre></tt>', '', regex=True)

        # pd.to_numeric(..., errors='coerce') замінює некоректні значення на NaN, якщо такі є.
        df["Year"] = pd.to_numeric(df["Year"], errors='coerce')
        df["Week"] = pd.to_numeric(df["Week"], errors='coerce')

        # Видаляє рядки з NaN у Year та Week, щоб уникнути помилок при подальших обчисленнях
        df.dropna(subset=["Year", "Week"], inplace=True)

        # Перетворює Year і Week на цілі числа (int):
        df["Year"] = df["Year"].astype(int)
        df["Week"] = df["Week"].astype(int)

        # Додаємо ID області у колонку "PROVINCE_ID"
        df["PROVINCE_ID"] = province_id

        # Видаляємо рядки з некоректними VHI та NaN
        df = df[df["VHI"] != -1].dropna()

        # Додаємо оброблені дані в загальний DataFrame
        combined_data = pd.concat([combined_data, df], ignore_index=True)

    except Exception as e:
        print(f"Помилка при читанні файлу {file_name}: {e}")

# Зберігаємо результат у файл
combined_data.to_csv(output_path, index=False)

# Виводимо перші 10 рядків
print("Об'єднані дані успішно збережено у", output_path)
print("Перші 10 рядків:")
print(combined_data.head(10))
print("Ось останні 10 рядків:")
print(combined_data.tail(10))


C:\Users\User\AppData\Local\Temp\ipykernel_5760\1226661203.py:51: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_data = pd.concat([combined_data, df], ignore_index=True)


Об'єднані дані успішно збережено у full.csv
Перші 10 рядків:
   Year Week    SMN     SMT    VCI    TCI    VHI PROVINCE_ID
0  1982    1  0.053  260.31  45.01  39.46  42.23           1
1  1982    2  0.054  262.29  46.83  31.75  39.29           1
2  1982    3  0.055  263.82  48.13  27.24  37.68           1
3  1982    4  0.053  265.33  46.09  23.91  35.00           1
4  1982    5  0.050  265.66  41.46  26.65  34.06           1
5  1982    6  0.048  266.55  36.56  29.46  33.01           1
6  1982    7  0.048  267.84  32.17  31.14  31.65           1
7  1982    8  0.050  269.30  30.30  32.50  31.40           1
8  1982    9  0.052  270.75  28.23  35.22  31.73           1
9  1982   10  0.056  272.73  25.25  37.63  31.44           1
Ось останні 10 рядків:
       Year Week    SMN     SMT    VCI    TCI    VHI PROVINCE_ID
59012  2024   43  0.259  281.45  79.71  17.45  48.58          27
59013  2024   44  0.229  279.41  76.74  13.33  45.04          27
59014  2024   45  0.206  278.07  77.64   8.70  43.

# Завдання 3

Реалізувати окрему процедуру, яка змінить індекси областей, які використані на
порталі NOAA (за англійською абеткою) на наступні, за українською (виключно
старі індекси на нові):

In [10]:
import pandas as pd

def update_province_ids(df):

    province_mapping = {
        1: 24,  2: 26,  3: 25,  4: 27,  5: 3,   6: 4,   7: 8,
        8: 21,  9: 22, 10: 23, 11: 10, 12: 9,  13: 11, 14: 12,
       15: 13, 16: 14, 17: 15, 18: 16, 19: 17, 20: 18, 21: 19,
       22: 20, 23: 6,  24: 1,  25: 2,  26: 7,  27: 5
    }

    # Переконуємося, що колонка "PROVINCE_ID" є у DataFrame
    if "PROVINCE_ID" not in df.columns:
        print("Колонка 'PROVINCE_ID' не знайдена у DataFrame!")
        return df

    # Оновлення значень
    df["PROVINCE_ID"] = df["PROVINCE_ID"].map(province_mapping)

    return df

# Оновлення індексів у `combined_data`
combined_data = update_province_ids(combined_data)

# Збереження у файл
output_file = "Updated_Provinces.csv"
combined_data.to_csv(output_file, index=False)

# Вивід результату
print(f"Оновлений файл збережено: {output_file}")
print(combined_data.head(10))


Оновлений файл збережено: Updated_Provinces.csv
   Year Week    SMN     SMT    VCI    TCI    VHI  PROVINCE_ID
0  1982    1  0.053  260.31  45.01  39.46  42.23           24
1  1982    2  0.054  262.29  46.83  31.75  39.29           24
2  1982    3  0.055  263.82  48.13  27.24  37.68           24
3  1982    4  0.053  265.33  46.09  23.91  35.00           24
4  1982    5  0.050  265.66  41.46  26.65  34.06           24
5  1982    6  0.048  266.55  36.56  29.46  33.01           24
6  1982    7  0.048  267.84  32.17  31.14  31.65           24
7  1982    8  0.050  269.30  30.30  32.50  31.40           24
8  1982    9  0.052  270.75  28.23  35.22  31.73           24
9  1982   10  0.056  272.73  25.25  37.63  31.44           24


# Завдання 4

### Реалізувати процедури для формування вибірок наступного виду (включаючи елементи аналізу):

o Ряд VHI для області за вказаний рік;

1. Отримати ряд VHI для області за вказаний рік

In [11]:
def get_vhi_for_region_year(df, province_id, year):

    return df[(df["PROVINCE_ID"] == province_id) & (df["Year"] == year)][["Year", "Week", "VHI"]]

vhi_data = get_vhi_for_region_year(combined_data, province_id=10, year=2020)
print(vhi_data)


       Year Week    VHI
23786  2020    1  37.78
23787  2020    2  38.41
23788  2020    3  39.74
23789  2020    4  41.90
23790  2020    5  43.53
23791  2020    6  43.37
23792  2020    7  41.50
23793  2020    8  39.53
23794  2020    9  38.90
23795  2020   10  39.07
23796  2020   11  38.66
23797  2020   12  38.60
23798  2020   13  37.73
23799  2020   14  35.97
23800  2020   15  34.35
23801  2020   16  33.99
23802  2020   17  35.73
23803  2020   18  38.44
23804  2020   19  42.71
23805  2020   20  47.25
23806  2020   21  48.13
23807  2020   22  47.18
23808  2020   23  45.82
23809  2020   24  46.15
23810  2020   25  48.05
23811  2020   26  49.25
23812  2020   27  51.20
23813  2020   28  53.44
23814  2020   29  55.37
23815  2020   30  56.30
23816  2020   31  54.89
23817  2020   32  52.78
23818  2020   33  48.81
23819  2020   34  45.37
23820  2020   35  43.23
23821  2020   36  41.38
23822  2020   37  40.84
23823  2020   38  39.22
23824  2020   39  37.70
23825  2020   40  37.41
23826  2020   41

In [12]:
# Функція для отримання індексу VHI (Vegetation Health Index) для певного регіону та року у заданому діапазоні тижнів
def get_vhi_for_region_year(df, province_id, year, start_week, end_week):
    # Фільтруємо DataFrame за заданими параметрами: область (PROVINCE_ID), рік (Year) і діапазон тижнів (Week)
    return df[
        (df["PROVINCE_ID"] == province_id) &  # Вибираємо рядки з потрібним ID області
        (df["Year"] == year) &  # Вибираємо рядки з потрібним роком
        (df["Week"] >= start_week) &  # Вибираємо тижні, що знаходяться у вказаному діапазоні
        (df["Week"] <= end_week)  # Включаємо верхню межу діапазону тижнів
    ][["Year", "Week", "VHI"]]  # Вибираємо тільки необхідні колонки для виводу

# Виклик функції для отримання VHI для області з ID=10 за 2020 рік у період з 15 по 20 тиждень
vhi_data = get_vhi_for_region_year(combined_data, province_id=10, year=2020, start_week=15, end_week=20)

# Виводимо результат
print(vhi_data)


       Year Week    VHI
23800  2020   15  34.35
23801  2020   16  33.99
23802  2020   17  35.73
23803  2020   18  38.44
23804  2020   19  42.71
23805  2020   20  47.25


2. Пошук екстремумів (min, max), середнього, медіани

In [13]:
def get_vhi_statistics(df, province_ids, years):

    # Фільтруємо дані за обраними областями та роками
    filtered_df = df[(df["PROVINCE_ID"].isin(province_ids)) & (df["Year"].isin(years))]

    # Групуємо за областю та роком, обчислюємо статистичні показники
    return filtered_df.groupby(["PROVINCE_ID", "Year"])["VHI"].agg(["min", "max", "mean", "median"]).reset_index()

# Виклик функції для областей з ID 5 і 10 у 2000 та 2020 роках
stats = get_vhi_statistics(combined_data, province_ids=[5, 10], years=[2000, 2020])
print(stats)


   PROVINCE_ID  Year    min    max       mean  median
0            5  2000  27.46  66.30  47.882885  47.375
1            5  2020  35.05  58.33  44.544231  43.660
2           10  2000  10.60  61.87  39.758269  35.915
3           10  2020  29.27  56.30  41.359423  39.375


3. Отримати ряд VHI за вказаний діапазон років для вказаних областей

In [14]:
def get_vhi_by_year_range(df, province_ids, start_year, end_year):

    result = df[(df["PROVINCE_ID"].isin(province_ids)) & (df["Year"].between(start_year, end_year))]

    if result.empty:
        print("Дані для вказаного діапазону відсутні!")

    return result[["Year", "Week", "PROVINCE_ID", "VHI"]]

# Використання:
province_ids = [3, 7, 12]  # Введи потрібні області
start_year = 2010  # Початковий рік
end_year = 2020  # Кінцевий рік
vhi_range_data = get_vhi_by_year_range(combined_data, province_ids, start_year, end_year)
print(vhi_range_data)


       Year Week  PROVINCE_ID    VHI
10150  2010    1            3  52.08
10151  2010    2            3  49.75
10152  2010    3            3  48.82
10153  2010    4            3  48.13
10154  2010    5            3  46.79
...     ...  ...          ...    ...
56623  2020   48            7  44.96
56624  2020   49            7  45.42
56625  2020   50            7  46.26
56626  2020   51            7  47.35
56627  2020   52            7  49.46

[1716 rows x 4 columns]


o Для всього набору даних виявити роки, протягом яких екстремальні
посухи торкнулися більше вказаного відсотка областей по Україні (20%
областей - 5 областей з 25). Повернути роки, назви областей з
екстремальними посухами та значення VHI;

4. Виявити роки, коли посуха торкнулася >20% областей (VHI < 15)

In [15]:
def find_drought_years(df, threshold=15, affected_percentage=0.2):

    province_count = df["PROVINCE_ID"].nunique()  # Загальна кількість областей
    min_affected = int(province_count * affected_percentage)  # Мінімальна кількість уражених областей

    drought_data = df[df["VHI"] < threshold].groupby(["Year"])["PROVINCE_ID"].nunique().reset_index()
    drought_years = drought_data[drought_data["PROVINCE_ID"] >= min_affected]

    if drought_years.empty:
        print("Не знайдено років, коли посуха торкнулася 20% областей.")
        return None

    # Отримуємо детальні дані про області, які потрапили під посуху
    drought_details = df[(df["Year"].isin(drought_years["Year"])) & (df["VHI"] < threshold)]

    return drought_details[["Year", "PROVINCE_ID", "VHI"]]

# Використання:
drought_info = find_drought_years(combined_data)
print(drought_info)


       Year  PROVINCE_ID    VHI
949    2000           24  14.64
950    2000           24  11.82
951    2000           24  10.81
952    2000           24  10.68
953    2000           24  12.30
...     ...          ...    ...
55932  2007            7  11.55
55933  2007            7  10.88
55934  2007            7  11.06
55935  2007            7  12.05
55936  2007            7  13.84

[88 rows x 3 columns]
